In [ ]:
import requests
import os
import json
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

#v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*vv*v*v   SET QUERY DETAILS   *v*v*v*v*v*v*v*v*v*v*v*v*v*v*vv*v*v*v*

search_url = "https://api.twitter.com/2/tweets/search/recent"
STOCK_TICKER = input('Input Stock Ticker:') #'GME' #or multiple: '(GME OR TSLA OR PNTR OR AMC)'
TWEET_QUERY = STOCK_TICKER + ' lang:en -is:retweet' #english only, exclude retweets
TWEET_FIELDS = 'id,text,created_at,public_metrics,entities,author_id' # if updating, also update output_fields
EXPANSIONS = 'author_id' # if updating, also update output_fields
output_fields = ['id','text','created_at','public_metrics.retweet_count','public_metrics.reply_count'\
,'public_metrics.like_count','public_metrics.quote_count','author_id','entities.cashtags'] #,'hf_sentiment','hf_score']

LAST_N_MINUTES = 10 # search tweets posted in last N minutes
MAXRESULTS = 100 # max results per query
MAXPAGES = 1 # max queries per run



cur_time_utc = datetime.utcnow().replace(microsecond=0)
start_time_utc = cur_time_utc-timedelta(minutes=LAST_N_MINUTES)
query_params = {'query': TWEET_QUERY, 'max_results': MAXRESULTS, 'start_time': start_time_utc.isoformat()+'Z', 'tweet.fields': TWEET_FIELDS, 'expansions': EXPANSIONS}

#^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*
#v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*vv*v*v  AUTHENTICATION  *v*v*v*v*v*v*v*v*v*v*v*v*v*v*vv*v*v*v*v*v*v*

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAO1PZQEAAAAAIzz5nD1OLo3XM%2Bx2zgx5B0H%2BoCU%3Dr4d9dQsizagR0nsXJYwsybY664VfdK02toetcIE0kljXS9L8pS'

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

#^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*
#v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*vv*v*v   GET RESPONSE   *v*v*v*v*v*v*v*v*v*v*v*v*v*v*vv*v*v*v*v*v*v*

def get_tweets(search_url,bearer_oauth,query_params):
    response = requests.get(search_url, auth=bearer_oauth, params=query_params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    if response.json()['meta']['result_count'] == 0:
        print('No tweets found')
        quit()
    return response.json()

dftweets = pd.DataFrame()
dfusers = pd.DataFrame()
page_count = 0
while True:
    tweets = get_tweets(search_url,bearer_oauth,query_params)
    page_count+=1
    dftweets = dftweets.append(pd.json_normalize(tweets['data']))
    dfusers = dfusers.append(pd.json_normalize(tweets['includes']['users']))
    if 'next_token' in tweets['meta'].keys() and page_count < MAXPAGES:
        query_params['next_token']=tweets['meta']['next_token']
    else:
        break

#^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*^*
#v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*v*vv*v*v   PRINT  *v*v*v*v*v*v*v*v*v*v*v*v*v*v*vv*v*v*v*v*v*v*v*v*v*v*
print('# pages:',page_count)
print('# tweets:',len(dftweets),'\n')

print(dftweets['text'])
#[print(tweet) for tweet in dftweets['text']]

# pages: 1
# tweets: 56 

0           @Jas0nYu You are not alone $TSLA $NIO $NVDA
1     LIMITED TIME OFFER: Get 5 FREE stocks valued u...
2     @garyblack00 Mr. Black $TSLA #GigaBerlin is go...
3     @squawksquare how abt $TSLA prove you wrong le...
4     $TSLA looking the strongest on the charts (of ...
5     @bill_selly @ICannot_Enough If FSD gets to L4/...
6     Retail traders did not chase the third day of ...
7     $TSLA and $QQQ bout to go down hard if they do...
8                         $TSLA crack 900 for nuts move
9                    Sold my weekly call on $TSLA. 875c
10    $KPRX red to green coming today.\nBottom bounc...
11    So $TSLA will become a defense stock; perfect ...
12    Here is #27 in the #EverythingsBetterWithFire ...
13    Tesla is still a bargain at $800 https://t.co/...
14    $AMD 100%\n$UPST 35%\n$BIDU 150 nw 120% (holdi...
15                   @TicTocTick But you like $tsla tho
16    $gnln penny baby! Could do numbers with volume...
17    ~I'm part of a s

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4bd80a34-af3c-4211-a433-b5e963efdc38' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>